# Importing

In [3]:
import datetime

from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

import pandas as pd
from pandas_datareader import data

from scipy.signal import savgol_filter
import statsmodels
import statsmodels.api as sm
import scipy.stats as stats
import scipy.optimize as optimize
import numpy as np
#from mpl_finance import candlestick_ohlc
import matplotlib.pyplot as plt
#For the usual plots!
#matplotlib inline
#Makes graphs in jupyter interactive!
%matplotlib widget

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'KODK', 'INTC', 'PANW']

#Removing warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
plt.close('all')

def stock_data(startd, endd, tickers):
    df = data.DataReader(tickers, 'yahoo', startd, endd).reset_index()
#df.set_index('Date', inplace=True)            <-- weggehaald zodat polyfit en backtest "i-1" werken
#uitrekenen van de daily return (cert)
    for ticker in tickers:
        df[('Return', ticker)] = df[('Close', ticker)].shift(1) - df[('Close', ticker)] 
    for ticker in tickers:
        df[('Percent_return', ticker)] = df[('Close', ticker)].pct_change()
    return df

def smoothing_sma_savgol(SMA, tickers):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        savgol_len = len(dfsma[(SMA + '_change', t)].dropna())
        if (savgol_len % 2) == 0:
            savgol_len += 1
            if (savgol_len % 2) != 0:
                return savgol_len
            else:
                continue
    #No real rules for polyorder, just use what fits best for its purpose!
        savgol = savgol_filter(dfsma[(SMA + '_change', t)].dropna(), savgol_len, polyorder=100)
        insert_len =  len(dfsma[(SMA + '_change', t)]) - len(savgol)
        savgol = np.insert(savgol, 0, np.zeros(insert_len) + np.nan)
        savgol = pd.Series(savgol, index=dfsma.index)
        dfsma[(SMA + '_change_smooth_savgol', t)] = savgol
    
def polyfit_df(tickers, df):
    dftemp = df.copy()
    for t in tickers:
        result = np.polyfit(dftemp[('Percent_return', t)].dropna().index, dftemp[('Percent_return', t)].dropna(), 50)
        poly_eq = np.poly1d(result)
        dftemp[('Polyfit', t)] = poly_eq(dftemp.index)
    dftemp = dftemp.reset_index()
    return dftemp

def smoothing_sma_sma(SMA, tickers):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        dfsma[(SMA + '_change_smooth_sma', t)] = dfsma[(SMA + '_change', t)].rolling(x).mean()
        
def smoothing_sma_ema(SMA, tickers):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        dfsma[(SMA + '_change_smooth_ema', t)] = dfsma[(SMA + '_change', t)].ewm(span=x, adjust=False).mean()

# Moving Averages

In [5]:
startd = datetime.datetime(2019, 1, 1) 
endd = datetime.datetime(2020, 1, 1)
df = stock_data(startd, endd, tickers)
dfsma = df.copy()   

for t in tickers:
    dfsma[('SMA10', t)] = dfsma[('Close', t)].rolling(10).mean()
    dfsma[('SMA20', t)] = dfsma[('Close', t)].rolling(20).mean()
    dfsma[('SMA50', t)] = dfsma[('Close', t)].rolling(50).mean()
    dfsma[('SMA100', t)] = dfsma[('Close', t)].rolling(100).mean()
    dfsma[('SMA200', t)] = dfsma[('Close', t)].rolling(200).mean()
    dfsma[('SMA10_change', t)] = dfsma[('SMA20', t)].pct_change()
    dfsma[('SMA20_change', t)] = dfsma[('SMA20', t)].pct_change()
    dfsma[('SMA50_change', t)] = dfsma[('SMA50', t)].pct_change()
    dfsma[('SMA100_change', t)] = dfsma[('SMA100', t)].pct_change()
    dfsma[('SMA200_change', t)] = dfsma[('SMA200', t)].pct_change()

#The fuction: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.savgol_filter.html
#Explanantion: https://stackoverflow.com/questions/20618804/how-to-smooth-a-curve-in-the-right-way
SMA = ['SMA10', 'SMA20', 'SMA50', 'SMA100', 'SMA200']
smoothing_sma_savgol(SMA[0], tickers) 
smoothing_sma_savgol(SMA[1], tickers)
smoothing_sma_savgol(SMA[2], tickers)
smoothing_sma_savgol(SMA[3], tickers)
smoothing_sma_savgol(SMA[4], tickers)
smoothing_sma_sma(SMA[0], tickers)
smoothing_sma_sma(SMA[1], tickers)
smoothing_sma_sma(SMA[2], tickers)
smoothing_sma_sma(SMA[3], tickers)
smoothing_sma_ema(SMA[0], tickers)
smoothing_sma_ema(SMA[1], tickers)
smoothing_sma_ema(SMA[2], tickers)

# Curve Fitting Time-sensitivity

Wat valt op:

-De percent_return geeft een beter return dan als ik SMA gebruik als basis! -> Check dit in DD detailed
    !!!!!

-Test polyfit op SMA. lijkt erop dat SMA30 het beste werkt qua time-sensitivity (hoe langer de SMA hoe accurater).
     
     IDEE: Miss  moet ik steeds dagen bij dezelfde fataframe optellen, maar dan de orginele output houden?/Steeds de return berekenen!

In [44]:
#15; + resid; 5

start_time = datetime.datetime.now().time()
print(start_time)
def polyfit_df(tickers, df):
    dftemp = df.copy()
    for t in tickers:
        result, resid, _, _, _ = np.polyfit(dftemp[('SMA_change', t)].dropna().index, dftemp[('SMA_change', t)].dropna(), 4, full=True)
        poly_eq = np.poly1d(result)
        #poly_eq = poly_eq + resid
        dftemp[('Polyfit', t)] = poly_eq(dftemp.index)
        dftemp[('Polyfit', t)] = dftemp[('Polyfit', t)]
    dftemp = dftemp.reset_index()
    return dftemp, resid

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']

startd = datetime.datetime(2016, 1, 1) 
#endd_1 = datetime.datetime(2019, 1, 1)
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()

for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('SMA', t)] = df[('Close', t)].rolling(5).mean()
    df[('SMA_change', t)] = df[('SMA', t)].pct_change()
    df[('SMA200', t)] = df[('Close', t)].rolling(200).mean()
    df[('SMA200_change', t)] = df[('SMA200', t)].pct_change()
df = df.dropna()
df = df.reset_index()

dfs = {'Polyfit' + str(i):df for i in range(250, df.index[-1]+1)}
for x in range(251, df.index[-1]+1):
    dict_key = 'Polyfit' + str(x)
    dfs[dict_key], resid = polyfit_df(tickers, df.iloc[:x])
    if x==300 or x==500:
        print(f'{x}:   {datetime.datetime.now().time()}')
        
end_time = datetime.datetime.now().time()
print(f'''
length of the last dataframe ({dict_key}) is: {len(dfs[dict_key])}
Time when finished:                   {end_time}''')

14:49:01.236330
300:   14:49:06.029439
500:   14:49:08.981478

length of the last dataframe (Polyfit805) is: 805
Time when finished:                   14:49:13.602289


In [45]:
plt.close('all')

df_polyfit_combined = df.copy()
for t in tickers:
    df_polyfit_combined[('Polyfit', t)] = 0
    for x in range(251, df.index[-1]+1): 
        dict_key = 'Polyfit' + str(x)
        df_polyfit_combined.loc[x, ('Polyfit', t)] = dfs[dict_key][('Polyfit', t)].iloc[-1]
######################CONTROL STRAT######################
for t in tickers:
    df[('Strat_control', t)] = df[('Percent_return', t)].apply(lambda x: 1)
df[('Percent_return', 'Total')] = 0
for t in tickers:
    df[('Percent_return', 'Total')].iloc[252:] += df[('Percent_return', t)].iloc[252:].cumsum()
######################CONTROL STRAT######################

i_length = range(251, df_polyfit_combined.index[-1])
for t in tickers:
        for i in i_length:
            if df_polyfit_combined.loc[i, ('Polyfit', t)]>=0.000 :
                df_polyfit_combined.loc[i+1, ('Strat_polyfit_6', t)] = 1
            if df_polyfit_combined.loc[i, ('Polyfit', t)]<0.000 :
                df_polyfit_combined.loc[i+1, ('Strat_polyfit_6', t)] = 0    
                
df_polyfit_combined[('Strat_polyfit_6_percent_return', 'Total')] = 0
for t in tickers:
    df_polyfit_combined[('Strat_polyfit_6_percent_return', t)] = df_polyfit_combined[('Strat_polyfit_6', t)] * df_polyfit_combined[('Percent_return', t)]
    df_polyfit_combined[('Strat_polyfit_6_percent_return', 'Total')] += df_polyfit_combined[('Strat_polyfit_6_percent_return', t)].cumsum()

plt.figure(figsize=(10,4))
plt.plot(df_polyfit_combined[('Strat_polyfit_6_percent_return', 'Total')])
plt.plot(df[('Percent_return', 'Total')].iloc[252:])
        
for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 1], 'hspace':0.02})
    ax[0].set_title(t)
    ax[0].plot(df_polyfit_combined[('Strat_polyfit_6_percent_return', t)].cumsum())
    ax[0].plot(df_polyfit_combined[('Percent_return', t)].iloc[252:].cumsum())
    ax[1].plot(df_polyfit_combined[('Strat_polyfit_6', t)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Savinsky-Golay Time-sensitivity

In [29]:
#13; 3; + resid

start_time = datetime.datetime.now().time()
print(start_time)
def smoothing_sma_savgol(tickers, df):
    dftemp = df.copy()
    for t in tickers:
        savgol_len = len(df[('SMA_change', t)].dropna())
        while (savgol_len % 2) == 0:
            savgol_len -= 1
    #No real rules for polyorder, just use what fits best for its purpose!
        savgol = savgol_filter(df[('SMA_change', t)].dropna(), savgol_len, polyorder=35)
        insert_len =  len(df[('SMA_change', t)]) - len(savgol)
        savgol = np.insert(savgol, 0, np.zeros(insert_len) + np.nan)
        dftemp[('Savgol', t)] = pd.Series(savgol, index=df.index)
        dftemp[('Savgol', t)] = dftemp[('Savgol', t)]
    dftemp.reset_index()
    return dftemp

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']

startd = datetime.datetime(2016, 1, 1) 
#endd_1 = datetime.datetime(2019, 1, 1)
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()

for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('SMA', t)] = df[('Close', t)].rolling(70).mean()
    df[('SMA_change', t)] = df[('SMA', t)].pct_change()
    df[('SMA200', t)] = df[('Close', t)].rolling(200).mean()
    df[('SMA200_change', t)] = df[('SMA200', t)].pct_change()
df = df.dropna()
df = df.reset_index()

dfs = {'Savgol' + str(i):df for i in range(250, df.index[-1]+1)}
for x in range(251, df.index[-1]+1):
    dict_key = ['Savgol' + str(x)]
    dfs['Savgol' + str(x)] = smoothing_sma_savgol(tickers, df.iloc[:x])
    #Time checkpoints
    if x==252 or x==500 or x==750:
        print(f'{x}:   {datetime.datetime.now().time()}')

end_time = datetime.datetime.now().time()
print(f'''
length of the last dataframe ({dict_key}) is: {len(dfs[dict_key])}
Time when finished:                   {end_time}''')

14:37:26.263994
252:   14:37:30.750860
500:   14:37:39.608799
750:   14:37:50.422927


AttributeError: 'datetime.time' object has no attribute 'str'

In [26]:
plt.close('all')

df_savgol_combined = df.copy()
for t in tickers:
    df_savgol_combined[('Savgol', t)] = 0
    for x in range(251, df.index[-1]+1): 
        #dict_key = 'Savgol' + str(x)
        df_savgol_combined.loc[x, ('Savgol', t)] = dfs['Savgol' + str(x)][('Savgol', t)].iloc[-1]
######################CONTROL STRAT######################
for t in tickers:
    df[('Strat_control', t)] = df[('Percent_return', t)].apply(lambda x: 1)
df[('Percent_return', 'Total')] = 0
for t in tickers:
    df[('Percent_return', 'Total')].iloc[252:] += df[('Percent_return', t)].iloc[252:].cumsum()
######################CONTROL STRAT######################

i_length = range(251, df_savgol_combined.index[-1])
for t in tickers:
        for i in i_length:
            if df_savgol_combined.loc[i, ('Savgol', t)]>=0.000 :
                df_savgol_combined.loc[i+1, ('Strat_savgol_6', t)] = 1
            if df_savgol_combined.loc[i, ('Savgol', t)]<0.000 :
                df_savgol_combined.loc[i+1, ('Strat_savgol_6', t)] = 0    
                
df_savgol_combined[('Strat_savgol_6_percent_return', 'Total')] = 0
for t in tickers:
    df_savgol_combined[('Strat_savgol_6_percent_return', t)] = df_savgol_combined[('Strat_savgol_6', t)] * df_savgol_combined[('Percent_return', t)]
    df_savgol_combined[('Strat_savgol_6_percent_return', 'Total')] += df_savgol_combined[('Strat_savgol_6_percent_return', t)].cumsum()

plt.figure(figsize=(10,4))
plt.plot(df_savgol_combined[('Strat_savgol_6_percent_return', 'Total')])
plt.plot(df[('Percent_return', 'Total')].iloc[252:])
        
for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 1], 'hspace':0.02})
    ax[0].set_title(t)
    ax[0].plot(df_savgol_combined[('Strat_savgol_6_percent_return', t)].cumsum())
    ax[0].plot(df_savgol_combined[('Percent_return', t)].iloc[252:].cumsum())
    ax[1].plot(df_savgol_combined[('Strat_savgol_6', t)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Ichi testing

In [7]:
tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']

startd = datetime.datetime(2016, 1, 1) 
#endd_1 = datetime.datetime(2019, 1, 1)
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()

for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('SMA', t)] = df[('Close', t)].rolling(70).mean()
    df[('SMA_change', t)] = df[('SMA', t)].pct_change()
    df[('SMA200', t)] = df[('Close', t)].rolling(200).mean()
    df[('SMA200_change', t)] = df[('SMA200', t)].pct_change()
df = df.dropna()
df = df.reset_index()

dfichi = df.copy().dropna() #werkt voor alle tickers!  -> Probleem verwijdert data voor alles als data ontbreekt
#dfichi.set_index('Date', inplace=True)

for t in tickers:
# Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
    nine_period_high = dfichi[('High', t)].rolling(window= 9).max()
    nine_period_low = dfichi[('Low', t)].rolling(9).min()
    dfichi[('Tenkan_sen', t)] = (nine_period_high + nine_period_low) /2
# Kijun-sen (Base Line): (26-period high + 26-period low)/2))
    period26_high = dfichi[('High', t)].rolling(26).max()
    period26_low = dfichi[('Low', t)].rolling(26).min()
    dfichi[('Kijun_sen', t)] = (period26_high + period26_low) / 2
# Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
    dfichi[('Senkou_span_a', t)] = ((dfichi[('Tenkan_sen', t)] + dfichi[('Kijun_sen', t)]) / 2).shift(26)
# Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
    period52_high = dfichi[('High', t)].rolling(52).max()
    period52_low = dfichi[('Low', t)].rolling(52).min()
    dfichi[('Senkou_span_b', t)] = ((period52_high + period52_low) / 2).shift(26)
# The most current closing price plotted 26 time periods behind (optional)
    dfichi[('Chikou_span', t)] = dfichi[('Close', t)].shift(-26)
###########Cloud tussen Senkou A en Senkou B    

In [8]:
plt.close('all')

######################CONTROL STRAT######################
for t in tickers:
    dfichi[('Strat_control', t)] = dfichi[('Percent_return', t)].apply(lambda x: 1)
dfichi[('Percent_return', 'Total')] = 0
for t in tickers:
    dfichi[('Percent_return', 'Total')].iloc[52:] += dfichi[('Percent_return', t)].iloc[52:].cumsum()
######################CONTROL STRAT######################

i_length = range(0, dfichi.index[-30])
for t in tickers:
        for i in i_length:
            if dfichi.loc[i, ('Close', t)]>=dfichi.loc[i, ('Kijun_sen', t)]:
                dfichi.loc[i+1, ('Strat_ichi', t)] = 1
            if dfichi.loc[i+25, ('Senkou_span_a', t)]<= dfichi.loc[i+25, ('Senkou_span_b', t)]:
                dfichi.loc[i+1, ('Strat_ichi', t)] = 0    
            else:
                dfichi.loc[i+1, ('Strat_ichi', t)] = 1
    ##Gebruik when .. then 1 untill ... then 0 -> save the current i and start again.
                
dfichi[('Strat_ichi_return', 'Total')] = 0
for t in tickers:
    dfichi[('Strat_ichi_return', t)] = dfichi[('Strat_ichi', t)] * dfichi[('Percent_return', t)]
    dfichi[('Strat_ichi_return', 'Total')] += dfichi[('Strat_ichi_return', t)].cumsum()

plt.figure(figsize=(10,4))
plt.plot(dfichi[('Strat_ichi_return', 'Total')])
plt.plot(dfichi[('Percent_return', 'Total')])
        
for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 1], 'hspace':0.02})
    ax[0].set_title(t)
    ax[0].plot(dfichi[('Strat_ichi_return', t)].cumsum())
    ax[0].plot(dfichi[('Percent_return', t)].cumsum())
    ax[1].plot(dfichi[('Strat_ichi', t)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Testing Stoploss

In [9]:
#SMA, 20; >=-0.01

start_time = datetime.datetime.now().time()
print(start_time)

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']

startd = datetime.datetime(2016, 1, 1) 
#endd_1 = datetime.datetime(2019, 1, 1)
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()

#Functions#
def wwma(values, n):
    return values.ewm(alpha=1/n, adjust=False).mean()

def Atr(data, t , n):
    high = data[('High', t)]
    low = data[('Low', t)]
    close = data[('Close', t)]
    data[('Tr0', t)] = abs(high - low)
    data[('Tr1', t)] = abs(high - close.shift())
    data[('Tr2', t)] = abs(low - close.shift())
    Tr = data[[('Tr0', t), ('Tr1', t), ('Tr2', t)]].max(axis=1)
    Atr = wwma(Tr, n)
    data[('Atr', t)] = Atr
    Stoploss = data[('Close', t)] - (data[('Atr', t)] * 2)
    return Stoploss 
#End Function#

for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('SMA', t)] = df[('High', t)].rolling(30).mean()
    df[('SMA_change', t)] = df[('SMA', t)].pct_change()
    
    df[('Stoploss', t)] = Atr(df, t ,len(df))
    
#   Do-untill code:  https://stackoverflow.com/questions/1662161/is-there-a-do-until-in-python
    x=0
    for i in df.index:
        while True:
            if df.loc[x:i, ('Atr', t)].max()<df.loc[i, ('Close', t)]:
                df.loc[i+1, ('Strat', t)] = 1
            if df.loc[x:i, ('Atr', t)].max()>=df.loc[i, ('Close', t)]:
                df.loc[i+1, ('Strat', t)] = 0
                x=i
                break
    
    #for i in df.index:
    #    if df.loc[i, ('Atr', t)]>df.loc[i, ('Close', t)]:
    #        df.loc[i+1, ('Strat', t)] = 1
    #    else:
    #        df.loc[i+1, ('Strat', t)] = 0
df[('Strat_return', 'Total')] = 0
for t in tickers:
    df[('Strat_return', t)] = df[('Strat', t)] * df[('Percent_return', t)]
    df[('Strat_return', 'Total')] += df[('Strat_return', t)].cumsum()
df = df.dropna()
df = df.reset_index()


for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 1], 'hspace':0.02})
    ax[0].set_title(t)
    #ax[0].plot(df[('Percent_return', t)], linewidth=0.5)
    ax[0].plot(df[('Close', t)], linewidth=0.5)
    ax[0].plot(df[('Stoploss', t)], linewidth=0.5)
    ax[1].plot(df[('Strat', t)], linewidth=0.5)
    #ax[0].plot(df[('Percent_return', t)].cumsum(), linewidth=1)
    #ax[0].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
    

end_time = datetime.datetime.now().time()
print(f'Time when finished:                   {end_time}')

01:53:19.544131


KeyboardInterrupt: 

# Achterhaalt

In [5]:
def polyfit_df(tickers, df):
    dftemp = df.copy()
    for t in tickers:
        result = np.polyfit(dftemp[('Percent_return', t)].dropna().index, dftemp[('Percent_return', t)].dropna(), 100)
        poly_eq = np.poly1d(result)
        dftemp[('Polyfit', t)] = poly_eq(dftemp.index)
    dftemp = dftemp.reset_index()
    return dftemp

dfpoly = polyfit_df(tickers, df)

i_length = dfpoly.index
for t in tickers:
        for i in i_length:
            if dfpoly.loc[i, ('Polyfit', t)]>=0:
                dfpoly.loc[i+1, ('Strat_polyfit_6', t)] = 1
            else:
                dfpoly.loc[i+1, ('Strat_polyfit_6', t)] = 0   
                
for i in i_length:
    for t in tickers:
           df[('Percent_return_cumsum', t)] = df[('Percent_return', t)].cumsum()

dfpoly[('Strat_polyfit_6_percent_return', t)] = 0
for i in i_length:
    for t in tickers:
        dfpoly[('Strat_polyfit_6_percent_return', t)] = dfpoly[('Strat_polyfit_6', t)] * dfpoly[('Percent_return', t)]
        dfpoly[('Strat_polyfit_6_percent_return', t)] = dfpoly[('Strat_polyfit_6_percent_return', t)].cumsum()

plt.figure(figsize=(8,3))
plt.plot(dfpoly[('Strat_polyfit_6_percent_return', 'INTC')])
plt.plot(df[('Percent_return_cumsum', 'INTC')], color='r')
print(dfpoly[('Strat_polyfit_6_percent_return', 'INTC')].iloc[-2])
print(df[('Percent_return_cumsum', 'INTC')].iloc[-1])
plt.figure(figsize=(8,3))
plt.plot(dfpoly[('Polyfit', 'SPY')])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.7359970764658881
0.2767154099351443


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
print(len(dfpoly1))
print(len(dfpoly3))
print(len(dfpoly3))
print(len(dfpoly4))
print(len(dfpoly5))
print(len(dfpoly6))
plt.figure(figsize=(10,5))
plt.plot(dfpoly1[('Polyfit', 'PANW')])
plt.plot(dfpoly2[('Polyfit', 'PANW')].iloc[-10:])
plt.plot(dfpoly3[('Polyfit', 'PANW')].iloc[474:])
plt.plot(dfpoly4[('Polyfit', 'PANW')].iloc[474:])
plt.plot(dfpoly5[('Polyfit', 'PANW')].iloc[486:])
plt.plot(dfpoly6[('Polyfit', 'PANW')].iloc[504:])
plt.plot(dfpoly6[('Polyfit', 'PANW')], color='black', linewidth=1)
plt.grid()

474
486
486
494
508
514


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …